In [4]:
from random import seed
from random import random
import numpy as np
 
# 네트워크 초기 설정
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network
 
seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [5]:
from math import exp #exp 계산에 대한 모듈 불러오기

#input에 대한 뉴런 활성화 계산
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] + inputs[i]  # 순전파 진행 
    return activation

def sigmoid(activation):
    return 1.0 / (1.0 + exp(-activation)) # 시그모이드 구현

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)  
            neuron['output'] = sigmoid(activation) # 나온 계산 값을 그대로 쓰나요?
            new_inputs.append(neuron['output']) # new_input은 다음 히든층에 들어갈 값이죠?
        inputs = new_inputs
    return inputs

**여기까지는 순전파 학습과정이었습니다. 이 과정이 끝나면 가중치가 바뀌나요?  
답변을 답변의 근거 코딩 결과와 함께 보여주세요.**

In [15]:
#이 과정이 끝나면 가중치가 바뀌진 않는다. 가중치는 오차 역전파를 통해서 바뀌게 될 것이다.

In [16]:
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)

[0.38709961854870767, 0.5296689516581048]


In [17]:
def sigmoid_derivative(output):
    return output * (1.0 - output) # 시그모이드 미분

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error) 
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output']) # 역전파시 오차는 어떻게 설정했나요?
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * sigmoid_derivative(neuron['output']) # 시그모이드 함수를 사용한 역전파 

In [18]:
expected = [0, 1]

backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [0.23098848185627577, 0.9846318794354859, 0.9009727614748673], 'output': 0.9575720296729436, 'delta': -0.003152162518359428}, {'weights': [0.1731706910306519, 0.38170581889404503, 0.3357617965908422], 'output': 0.8688282927318582, 'delta': 0.004037422355439106}]
[{'weights': [-0.5232815547008197, -0.5338876720149377, -1.2287514363762204], 'output': 0.38709961854870767, 'delta': -0.09184074084670625}, {'weights': [-1.0723408240348868, -0.11612303770626536, -0.519121073721081], 'output': 0.5296689516581048, 'delta': 0.11716875473579505}]


In [19]:
def weights_update(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta'] * inputs[j] # 퍼셉트론 학습 규칙
            

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row) # 순전파 
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))]) 
            backward_propagate_error(network, expected)
            weights_update(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [20]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

In [30]:
n_inputs = len(dataset[0]) -1 # 뉴럴렛의 입력노드로 뭐가 들어가죠? 그럼 입력 노드의 개수는?
n_outputs = len(set([row[-1] for row in dataset])) # 뉴럴렛의 출력노드의 개수는 뭐라고 했죠? 
network = initialize_network(n_inputs, 2, n_outputs)

train_network(network, dataset, 0.13, 100, n_outputs) # 자유롭게 설정하고 최적을 찾아보세요.

# 학습된(최적화)된 네트워크가 초기 네트워크와 달리 어떻게 변하였는지 출력하시오. (layer별로,hint : for문))
predictions = list()

for row in test:
    prediction = predict(network, row)
    predictions.append(prediction)
return(predictions)

>epoch=0, lrate=0.130, error=9.655
>epoch=1, lrate=0.130, error=9.643
>epoch=2, lrate=0.130, error=9.630
>epoch=3, lrate=0.130, error=9.616
>epoch=4, lrate=0.130, error=9.601
>epoch=5, lrate=0.130, error=9.584
>epoch=6, lrate=0.130, error=9.566
>epoch=7, lrate=0.130, error=9.547
>epoch=8, lrate=0.130, error=9.525
>epoch=9, lrate=0.130, error=9.502
>epoch=10, lrate=0.130, error=9.476
>epoch=11, lrate=0.130, error=9.447
>epoch=12, lrate=0.130, error=9.415
>epoch=13, lrate=0.130, error=9.379
>epoch=14, lrate=0.130, error=9.338
>epoch=15, lrate=0.130, error=9.292
>epoch=16, lrate=0.130, error=9.239
>epoch=17, lrate=0.130, error=9.179
>epoch=18, lrate=0.130, error=9.109
>epoch=19, lrate=0.130, error=9.028
>epoch=20, lrate=0.130, error=8.934
>epoch=21, lrate=0.130, error=8.824
>epoch=22, lrate=0.130, error=8.696
>epoch=23, lrate=0.130, error=8.549
>epoch=24, lrate=0.130, error=8.382
>epoch=25, lrate=0.130, error=8.200
>epoch=26, lrate=0.130, error=8.010
>epoch=27, lrate=0.130, error=7.824
>e

NameError: name 'test' is not defined

In [31]:
# 학습한 네트워크로 예측값을 뽑아보자.

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs)) # 순전파 결과에서 어떤것이 최종 아웃풋이 되나요?

In [32]:
# 네트워크가 잘 학습되었는지 확인해보자. 

for row in dataset:
    prediction = predict(network, row) # 앞서 최적(학습)시킨 네트워크로 잘 학습되었는지 평가 
    print('실제값=%d, 예측값=%d' % (row[-1], prediction))

실제값=0, 예측값=1
실제값=0, 예측값=1
실제값=0, 예측값=1
실제값=0, 예측값=1
실제값=0, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
